In [ ]:
#CSE 575 Group 2 Project 1
#Summer 2024
#Prof. Samira Ghayekhloo
#Project Team: Arjun Dadhwal, Ching-Chun Yuan, Jeffrey Li

import scipy.io # Library used to load the datasets.
import numpy as np # Library used for various operations involving arrays of various dimensions.
from google.colab import drive #This was used to import the files stored on Google Drive. Feel free to remove these two lines.
drive.mount('/drive')


#Given tasks:
    # 1. Write code to extract features for both training set and testing set.
    # 2. Write code to implement the Naive Bayes Classifier and use it produce a predicted label for each testing sample.
    # 3. Write code to compute the classification accuracy, for the Naive Bayes Classifier
    # 4. Write a short report summarizing the results, including the final classification accuracy.


#Loading the datasets.
#Note: Feel free to change the path to the files based on your setup.

train_0_file = scipy.io.loadmat('/drive/MyDrive/test/train_0_img.mat') #Loading the training data set with digit 0 samples.
train_1_file = scipy.io.loadmat('/drive/MyDrive/test/train_1_img.mat') #Loading the training data set with digit 1 samples.
test_0_file = scipy.io.loadmat('/drive/MyDrive/test/test_0_img.mat')  #Loading the testing data set with digit 0 samples.
test_1_file = scipy.io.loadmat('/drive/MyDrive/test/test_1_img.mat') #Loading the testing data set with digit 1 samples.

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
#The below variables extract the relevant data containing arrays from the datasets which were of "dict type", using the "target_img" key.

train_data_0 = train_0_file['target_img']
train_data_1 = train_1_file['target_img']
test_data_0 = test_0_file['target_img']
test_data_1 = test_1_file['target_img']



In [ ]:
#The code below accomplishes the given Task 1.

#This function extracts 2 features from the given samples of the dataset passed through parameter. The samples are images represented by a 2-D array of pixels with brightness values.
#It then returns a Feature array containing the value of the features for each sample.
# Feature 1 = Mean of the brightness of the pixels in each sample.
# Feature 2 = Standard deviation of the brightness of the pixels in each sample.

def extractFeature(images):
    num_images = images.shape[2] # The size of the third dimension of the data array gives the total number of sample images.
    features = np.zeros((num_images, 2)) # Create the 2-D array to store the value of the features for each sample.
    for i in range(num_images):
        img = images[:, :, i] # Extract the brightness values of all the pixels of the current image.
        features[i, 0] = np.mean(img) # Store the first feature, the mean brightness.
        features[i, 1] = np.std(img) # Store the second feature, the standard deviation.
    return features

In [ ]:
#Extract the features from the training datasets, for digit 0 and digit 1.

train_0feat = extractFeature(train_data_0)
train_1feat = extractFeature(train_data_1)

#Extract the features from the testing datasets, for digit 0 and digit 1.

test_0feat = extractFeature(test_data_0)
test_1feat = extractFeature(test_data_1)

#Calculate the total number of samples from the training dataset for each digit. This will be total number of that particular digit in the sample.

size_of_train_img0  = train_0feat.shape[0]
size_of_train_img1  = train_1feat.shape[0]

#Calculate the prior probabilities for each digit in the training set. This will be useful later on in the Naive Bayes function.
#In Naive Bayes, we can predict the Prior probability with the formula: P(Y) = (The number of samples belonging to Y class) / Total number of samples)

prior0 = (size_of_train_img0) / (size_of_train_img0 + size_of_train_img1) #The total size of training dataset is the sum of the sizes of both the dataset.
prior1 = (size_of_train_img1) / (size_of_train_img0 + size_of_train_img1)


In [ ]:
#This function takes in features of the training set of a digit as its input and uses that to estimate the parameters for the 2-D normal distribution for that digit.
#The estimated parameters are the Mean vector for both the features in the dataset and the Covariance matrix for both the features in the dataset.

def getParameters(features):
    mean = np.mean(features, axis=0) # Calculate the mean vector of the features.
    cov_matrix = np.cov(features, rowvar = False) # Calculate the Covariance.
    return mean, cov_matrix

#The estimated paramters for the training sets for both the digits, 0 and 1.

train_0_mean, train_0_covMatrix = getParameters(train_0feat)
train_1_mean, train_1_covMatrix = getParameters(train_1feat)



In [ ]:
#The 2D Gaussian (Normal) Distribution is used to calculate the Likelihood P(x|Y),where x is the feature and Y is the label.
#The likelihood is useful for later on in the Naive Bayes classifier for calculating the Posterior Probability which will be used for predicting the class labels.
#Formula: p(x) = (1/( (2pi)^(d/2) * sqrt(covariance matrix))) * exp(-1/2(x - mean)^t * inverse(covariance matrix) * (x - mean))


#x is the current testing set sample. mu the mean vector and sigma the covariance matrix which are the earlier calculated parameters for the given digit from the normal distributions of its training set.
def Gaussian2D(x, mu, sigma):
    d = x.shape[1]  #d is the number of features, which will be 2.
    diff = x - mu # Calculate the difference between the current sample and
    inv_sigma = np.linalg.inv(sigma) #Calculate the inverce of the Covariance matrix.
    norm_const = 1.0 / (np.sqrt((2 * np.pi) ** d * np.linalg.det(sigma))) # Calculate the denominator for the Gaussian formula.
    result = norm_const * np.exp(-0.5 * np.sum(diff @ inv_sigma * diff, axis=1))
    return result

In [ ]:
#A Gaussian Naives Bayes classifier. Takes in the features for each sample in the testing data and using previously established parameters
#predicts the label for the sample.

def naiveBayes(test_samples, mean_0, cov_7, mean_8, cov_8, prior0, prior1):

    #Calculate posterior probability for both the labels 0 and 1.
    #Posterior Probability (P(Y|X)) = Likelihood (P(X|Y)) * Prior P(Y) where Y is the label, X is the features vector for the sample.
    #The likelihood is calculated using the Gaussian Distribution Formula using the earlier created function.

    posterior0 = Gaussian2D(test_samples, mean_0, cov_7) * prior0 #For digit 0. Function uses the features vector and the estimated training set parameters along with prior.
    posterior1 = Gaussian2D(test_samples, mean_8, cov_8) * prior1 #For digit 0. Function uses the features vector and the estimated training set parameters along with prior.


    #It returns true (1) if the posterior for digit 0 is higher, which means that 0 is the predicted digit.

    return posterior0 > posterior1

In [ ]:
test_features = np.vstack([test_0feat, test_1feat]) #Combine the feature values for the samples in the testing data sets for both digits into one stack.

#Generate the labels for the testing set for comparision with the predictions. Since we already know the digits for the testing set, generate the labels based on the number of samples.
#True(1) if the label is digit 0 or False (0) if the label is digit 1.
test_labels = np.concatenate([np.ones(test_0feat.shape[0]), np.zeros(test_1feat.shape[0])])

#Call the Naive Bayes function to predict the labels from testing data set and store the result in the predictions array.
#Note: Predictions store whether that sample should be digit 0 or not. So digit 0 predictions will be stored as 1 (true).

predictions = naiveBayes(test_features, train_0_mean, train_0_covMatrix, train_1_mean, train_1_covMatrix, prior0, prior1)

#This functions takes in the predictions made by the Naive Bayes classifier and compares them with the target labels in the test set based on the target class.
#Note: The labels for digit 0 are stored as 1 (true).

def accuracy(predictions, labels, isDigit0):
    target_mask = labels == isDigit0 #Get the labels for the target digit that we are trying to find.
    correct = np.sum(predictions[target_mask] == labels[target_mask]) #Count the total number of correct predictions i.e. prediction is same as the target class from the training set.
    total = np.sum(target_mask) # Get the total number of the samples for which the labels was checked.
    return correct/total # Return the accuracy as the ratio of correctly predicted labels to the total number of labels.

#Calculate the classification accuracy for the predictions made for each digit.

accuracy0 = accuracy(predictions, test_labels, 1)
accuracy1 = accuracy(predictions, test_labels, 0)

#Print the results for the classification accuracy for each digit.

print(f'Accuracy for digit 0: {accuracy0 * 100:.2f}%')
print(f'Accuracy for digit 1: {accuracy1 * 100:.2f}%')

Accuracy for digit 0: 97.45%
Accuracy for digit 1: 96.39%
